# Descarga de Dados GFS


Trabajaremos con el utilitario [siphon](https://unidata.github.io/siphon/latest/index.html), que fue diseñado para tener acceso directo a al servidor [THREDDS](https://www.unidata.ucar.edu/software/tds/current/)

Para utilizar el `siphon` debemos tener en consideración lo siguiente:
- Python >= 3.5
- numpy >= 1.8
- pandas

Y otras informaciones que podemos encontrar en el manual de instalación de la herramienta.


## Instalación de librerias necesarias

- Librería siphon


```bash
conda install -c conda-forge siphon xarray[complete] netCDF4
```

>El `netCDF4` será instalado para ser utilizado como background del siphon a la hora de hacer los requerimientos \
En este caso iremos a instalar el `xarray` para tener un control facilitado de los archivos `netcdf`



La lista completa de datos como medio grado entra en este [link](https://thredds.ucar.edu/thredds/catalog/catalog.html)

In [2]:
from siphon.catalog import TDSCatalog

# si el catalogo que se desea es de 0.5 grados
# best_gfs = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p25deg/catalog.xml')

# si el catalogo que se desea es de 0.25 grados
best_gfs = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p25deg/catalog.xml')

# lista de colecciones
best_gfs.datasets

['Full Collection (Reference / Forecast Time) Dataset', 'Best GFS Quarter Degree Forecast Time Series', 'Latest Collection for GFS Quarter Degree Forecast']

## Verificando datos disponibles

In [9]:
best_disp = list(best_gfs.datasets.values())[0]
ncss = best_disp.subset()
query = ncss.query()
print(ncss.variables)

{'Water_runoff_surface_Mixed_intervals_Accumulation', 'Total_ozone_entire_atmosphere_single_layer', 'Total_cloud_cover_convective_cloud', 'Snow_mixing_ratio_hybrid', 'Vertical_velocity_geometric_isobaric', 'Total_cloud_cover_isobaric', 'Convective_precipitation_surface_Mixed_intervals_Accumulation', 'v-component_of_wind_planetary_boundary', 'u-component_of_wind_pressure_difference_layer', 'Field_Capacity_surface', 'u-component_of_wind_isobaric', 'u-component_of_wind_altitude_above_msl', 'Pressure_surface', 'Ice_water_mixing_ratio_hybrid', 'Graupel_snow_pellets_isobaric', 'Rain_mixing_ratio_hybrid', 'Reflectivity_hybrid', 'Haines_index_surface', 'u-component_of_wind_tropopause', 'Specific_humidity_isobaric', 'Upward_Short-Wave_Radiation_Flux_surface_Mixed_intervals_Average', 'ICAO_Standard_Atmosphere_Reference_Height_tropopause', 'Categorical_Ice_Pellets_surface', 'Relative_humidity_entire_atmosphere_single_layer', 'Vertical_Speed_Shear_potential_vorticity_surface', 'u-component_of_wind

In [4]:
best_ds = list(best_gfs.datasets.values())[1]

# diccionario de urls disponibles con la información
best_ds.access_urls

{'OPENDAP': 'https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/GFS/Global_0p25deg/Best',
 'CdmRemote': 'https://thredds.ucar.edu/thredds/cdmremote/grib/NCEP/GFS/Global_0p25deg/Best',
 'JupyterNotebook': 'https://thredds.ucar.edu/thredds/notebook/grib/NCEP/GFS/Global_0p25deg/Best',
 'NetcdfSubset': 'https://thredds.ucar.edu/thredds/ncss/grid/grib/NCEP/GFS/Global_0p25deg/Best',
 'WMS': 'https://thredds.ucar.edu/thredds/wms/grib/NCEP/GFS/Global_0p25deg/Best',
 'WCS': 'https://thredds.ucar.edu/thredds/wcs/grib/NCEP/GFS/Global_0p25deg/Best',
 'ISO': 'https://thredds.ucar.edu/thredds/iso/grib/NCEP/GFS/Global_0p25deg/Best',
 'NCML': 'https://thredds.ucar.edu/thredds/ncml/grib/NCEP/GFS/Global_0p25deg/Best',
 'UDDC': 'https://thredds.ucar.edu/thredds/uddc/grib/NCEP/GFS/Global_0p25deg/Best'}

In [5]:
# utilizando el servicio de netcdf-subset

from siphon.ncss import NCSS
ncss = NCSS(best_ds.access_urls['NetcdfSubset'])

In [6]:
# generando un sistema de consulta en el netcdf-subset
query = ncss.query()

In [7]:
from datetime import datetime

now = datetime.utcnow()
# buscando la posición de lima
query.lonlat_box(west=-80, north=0, east=-65, south=-20).vertical_level(100000).time(now)
query.accept('netcdf4')
query.variables('Temperature_isobaric', 'Relative_humidity_isobaric')

var=Relative_humidity_isobaric&var=Temperature_isobaric&time=2022-10-24T21%3A30%3A26.675720&west=-80&east=-65&south=-20&north=0&vertCoord=100000&accept=netcdf4

In [8]:
data = ncss.get_data(query)
list(data.variables.keys())

['reftime',
 'latitude',
 'time',
 'isobaric',
 'longitude',
 'Relative_humidity_isobaric',
 'Temperature_isobaric',
 'LatLon_721X1440-0p13S-180p00E-2']